In [1]:
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import copy

In [2]:
def load_data():
    data = loadmat("mnist_all.mat")

    # print(data.keys())

    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for i in range(10):
        temp_df = pd.DataFrame(data["train" + str(i)])
        temp_df['label'] = i
        train_data = train_data.append(temp_df)
        temp_df = pd.DataFrame(data["test" + str(i)])
        temp_df['label'] = i
        test_data = test_data.append(temp_df)

    train_data = shuffle(train_data)
    test_data = shuffle(test_data)

    train_labels = np.array(train_data['label'])
    test_labels = np.array(test_data['label'])

    train_data = train_data.drop('label', axis=1)
    test_data = test_data.drop('label', axis=1)

    train_data = np.array(train_data) / 255
    test_data = np.array(test_data) / 255

    return train_data, test_data, train_labels, test_labels

In [3]:
BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x_train, x_test, y_train, y_test = load_data()

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.2, random_state=0)

In [4]:
class MNIST_DATASET(Dataset):
    def __init__(self, x, y, phase):
        self.x = x
        self.phase = phase

        self.y = y

        self.transforms = transforms.Compose([
            transforms.ToTensor()
        ])

    def __getitem__(self, index):
        return self.transforms(self.x[index].reshape(28, 28)), self.y[index]

    def __len__(self):
        return len(self.x)

In [5]:
class MNIST_LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_layers, num_classes):
        super(MNIST_LSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=in_dim, hidden_size=hidden_dim, num_layers=num_layers)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):

        # hidden = (torch.randn(2, 256, 256).double(),
        #           torch.randn(2, 256, 256).double())

        out, hidden = self.lstm(x)
        out = out[-1, :, :]
        x = self.classifier(out)
        return x

In [6]:
def run():    
    model = MNIST_LSTM(in_dim=28, hidden_dim=256, num_layers=2, num_classes=10)
    model.to(DEVICE)
    model.double()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_dataset = MNIST_DATASET(x_train, y_train, 'train')
    valid_dataset = MNIST_DATASET(x_valid, y_valid, 'valid')
    test_dataset = MNIST_DATASET(x_test, y_test, 'test')

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

    valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

    best_model = None
    best_acc = -1

    for epoch in range(EPOCHS):
        model.train()
        for index, (images, labels) in enumerate(train_loader):

            images = torch.squeeze(images)
            images = images.permute(1, 0, 2).double()

            images = images.to(DEVICE)

            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if index % 100 == 0:
                print('Train Epoch [{}/{}], Loss: {:.4f}'
                      .format(epoch + 1, EPOCHS, loss.item()))

        model.eval()
        with torch.no_grad():
            valid_correct = 0
            for images, labels in valid_loader:

                images = torch.squeeze(images)
                images = images.permute(1, 0, 2).double()

                images = images.to(DEVICE)
                labels = labels.to(DEVICE)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)

                valid_correct += (predicted == labels).sum().item()

            acc = 100.0 * valid_correct / len(valid_loader.dataset)

            print("Epoch: {} The accuracy of total {} images: {}%".format(epoch + 1, len(valid_loader.dataset),
                                                                      100.0 * valid_correct / len(valid_loader.dataset)))
            if acc > best_acc:
                best_acc = acc
                best_model = copy.deepcopy(model)
                print('get new model!')

    model = copy.deepcopy(best_model)
    model.to(DEVICE)
    model.double()
    with torch.no_grad():
        test_correct = 0
        for images, labels in test_loader:
            images = torch.squeeze(images)
            images = images.permute(1, 0, 2).double()

            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images)

            _, predicted = torch.max(outputs, 1)

            test_correct += (predicted == labels).sum().item()

    print("The accuracy of total {} images: {}%".format(len(test_loader.dataset),
                                                                  100.0 * test_correct / len(test_loader.dataset)))


In [7]:
run()

Train Epoch [1/10], Loss: 2.3024
Train Epoch [1/10], Loss: 0.3389
Epoch: 1 The accuracy of total 12000 images: 94.90833333333333%
get new model!
Train Epoch [2/10], Loss: 0.1359
Train Epoch [2/10], Loss: 0.1409
Epoch: 2 The accuracy of total 12000 images: 97.61666666666666%
get new model!
Train Epoch [3/10], Loss: 0.0746
Train Epoch [3/10], Loss: 0.0737
Epoch: 3 The accuracy of total 12000 images: 98.04166666666667%
get new model!
Train Epoch [4/10], Loss: 0.0526
Train Epoch [4/10], Loss: 0.1112
Epoch: 4 The accuracy of total 12000 images: 98.20833333333333%
get new model!
Train Epoch [5/10], Loss: 0.0334
Train Epoch [5/10], Loss: 0.0503
Epoch: 5 The accuracy of total 12000 images: 98.375%
get new model!
Train Epoch [6/10], Loss: 0.0140
Train Epoch [6/10], Loss: 0.0784
Epoch: 6 The accuracy of total 12000 images: 98.34166666666667%
Train Epoch [7/10], Loss: 0.0308
Train Epoch [7/10], Loss: 0.0184
Epoch: 7 The accuracy of total 12000 images: 98.7%
get new model!
Train Epoch [8/10], Loss